## 首次创建<规范表>的范式

In [1]:
import os, re, calendar
import numpy as np
import pandas as pd 
from pypinyin import Style, pinyin

In [2]:
# 工具函数
def re_subFN_sql(file_list):
    '''
    筛选子目录下的所有".sql"的文件名
    file_list: 路径下包含的各文件名
    '''
    target_files = []
    for i, name in enumerate(file_list):
        if name.find('.sql')>=0:
            target_files.append(name)
    
    return target_files

def eng_name(s):
    '''
    取出SQL语句中字段英文名
    '''
    slist = s.split(" ")
    for i in slist:
        if i != "":
            return i
    return "字符串为空"

def trim(lis):
    '''
    截取SQL文件中定义字段语句
    '''
    i=0
    for j in lis:
        if j.split(' ')[0] == 'CREATE':
            return lis[i+2:]
        i += 1

In [3]:
file_names = re_subFN_sql(os.listdir(os.path.abspath(os.path.dirname(os.getcwd()))))
os.chdir(os.path.abspath(os.path.dirname(os.getcwd())))

df = pd.DataFrame(columns=['英文名','SQL语句'])
for name in file_names:

    sql = open(name, "r", encoding = "utf8")
    sqltxt = sql.readlines()# 此时 sqltxt 为 list 类型

    # 读取之后关闭文件
    sql.close()
    
    sqltxt = trim(sqltxt[:-2])
    for i in sqltxt:
        try:
            key =  re.findall("COMMENT '(.*?)'",i)[0]
            #df.loc[key] = [eng_name(i),i]  // for windows
            df.loc[key] = [eng_name(i).replace('`',''),i]
        except:
            #print(name,i)  // 输出异常信息
            pass    
        
# 存储<规范表>文件
df.to_excel('业务数据字段命名及设计规范.xlsx')

The kener below may not be execute

In [4]:
# 调用<规范表>的演示
df = pd.read_excel('业务数据字段命名及设计规范.xlsx',index_col=0)
df.head()

,英文名,SQL语句
乐观锁,REVISION,`REVISION` int(11) NULL DEFAULT NULL COMMENT...
统计时间,DATA_DATE,`DATA_DATE` date NULL DEFAULT NULL COMMENT '...
分公司代码,COMPANY_CODE,`COMPANY_CODE` varchar(32) CHARACTER SET utf...
分公司名称,COMPANY_NAME,`COMPANY_NAME` varchar(128) CHARACTER SET ut...
部门组代码,DEPARTMENT_GROUP,`DEPARTMENT_GROUP` varchar(128) CHARACTER SE...


### 我们之后仅需对<规范表>进行维护即可创建自动数据表